In [5]:
import glob

extensions = ("*.png", "*.jpg", "*.jpeg")

images = []
for ext in extensions:
    images.extend(sorted(glob.glob(ext)))

if not images:
    raise RuntimeError("No images found in current directory")

AUTO_ADVANCE_MS = 5000
ANIM_MS = 400

# Build JS arrays
images_js = "[" + ",".join(f"'{img}'" for img in images) + "]"
urls_js = "[" + ",".join("'http://meritpages.com/michael_woodcock'" for _ in images) + "]"

html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>Awards Slideshow</title>

<style>
  body {{
    font-family: Arial, sans-serif;
    background: #121212;
    color: #e0e0e0;
    margin: 0;
    overflow: hidden;
  }}

  .back-button {{
    position: fixed;
    top: 1rem;
    left: 1rem;
    display: flex;
    align-items: center;
    gap: 0.5rem;
    background: rgba(30, 30, 30, 0.9);
    border: 1px solid rgba(0, 255, 255, 0.2);
    border-radius: 999px;
    padding: 0.4rem 0.9rem;
    color: #ffffff;
    font-size: 1.1rem;
    cursor: pointer;
    z-index: 10000;
    transition: all 0.3s ease;
    box-shadow: 0 0 6px rgba(0, 255, 255, 0.3);
  }}

  .back-button:hover {{
    background: rgba(0, 255, 255, 0.15);
    color: #00bcd4;
    transform: scale(1.05);
    box-shadow: 0 0 10px rgba(0, 255, 255, 0.6);
  }}

  /* FULL-VIEWPORT SLIDESHOW */
  .slideshow {{
    position: relative;
    width: 100vw;
    height: 100vh;
    display: flex;
    align-items: center;
    justify-content: center;
    overflow: hidden;
  }}

  /* SAFE IMAGE AREA — NEVER OVERLAPS UI */
  .slide {{
    position: absolute;
    top: 0;
    left: 0;
    width: calc(100vw - 20px);      /* 10px padding left/right */
    height: calc(100vh - 120px);    /* compact UI zone */
    padding: 10px;                  /* guaranteed padding */
    box-sizing: border-box;
    display: flex;
    align-items: center;
    justify-content: center;
    transition: transform {ANIM_MS}ms ease;
  }}

  /* Image ALWAYS fits inside padded safe area */
  .slide img {{
    width: 100%;
    height: 100%;
    object-fit: contain;
    border-radius: 6px;
    box-shadow: 0 20px 60px rgba(0,0,0,0.7);
    display: block;
    cursor: zoom-in;
    transition: transform 0.3s ease;
  }}

  .slide img.zoomed {{
    transform: scale(1.8);
    cursor: zoom-out;
  }}

  .off-right {{ transform: translateX(100vw); }}
  .off-left  {{ transform: translateX(-100vw); }}
  .center    {{ transform: translateX(0); }}

  /* COMPACT CONTROLS */
  .controls {{
    position: fixed;
    bottom: 70px;
    width: 100%;
    display: flex;
    justify-content: center;
    gap: 10px;
    z-index: 1000;
  }}

  button.control {{
    background: #1f2937;
    color: #e5e7eb;
    border: 1px solid #374151;
    padding: 6px 12px;
    font-size: 13px;
    border-radius: 6px;
    cursor: pointer;
  }}

  .counter {{
    position: fixed;
    bottom: 50px;
    width: 100%;
    text-align: center;
    font-size: 12px;
    color: #9ca3af;
    z-index: 1000;
  }}

  /* THUMBNAILS — NO SCROLLING EVER */
  .thumb-strip {{
    position: fixed;
    bottom: 0;
    width: 100%;
    height: 50px;
    background: rgba(0,0,0,0.4);
    display: flex;
    justify-content: center;
    align-items: center;
    gap: 6px;
    padding: 4px;
    box-sizing: border-box;
    overflow: hidden; /* NO SCROLLING */
    flex-wrap: wrap;  /* wrap instead of scroll */
  }}

  .thumb-strip img {{
    height: 40px;
    border-radius: 4px;
    cursor: pointer;
    opacity: 0.6;
    transition: opacity 0.2s, transform 0.2s;
  }}

  .thumb-strip img.active {{
    opacity: 1;
    transform: scale(1.05);
    border: 2px solid cyan;
  }}
</style>
</head>

<body>

<button class="back-button" onclick="history.back()">
  <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">
    <path d="M15 18l-6-6 6-6"/>
  </svg>
  <span>Back</span>
</button>

<div class="slideshow"></div>

<div class="controls">
  <button class="control" onclick="prev()">⟵ Previous</button>
  <button class="control" onclick="toggleAuto()" id="toggle">⏸ Pause</button>
  <button class="control" onclick="nextSlide()">Next ⟶</button>
</div>

<div class="counter" id="counter"></div>

<div class="thumb-strip" id="thumbs"></div>

<script>
// Auto-generated by Python
const images = {images_js};
const urls = {urls_js};

let index = 0;
let auto = true;
let timer = null;

const slideshow = document.querySelector(".slideshow");
const counter = document.getElementById("counter");
const toggleBtn = document.getElementById("toggle");
const thumbs = document.getElementById("thumbs");

function resetTimer() {{
  if (timer) clearInterval(timer);
  startAuto();
}}

function updateCounter() {{
  counter.textContent = `${{index + 1}} / ${{images.length}}`;
}}

function updateThumbnails() {{
  [...thumbs.children].forEach((t, i) => {{
    t.classList.toggle("active", i === index);
  }});
}}

// Build thumbnails
images.forEach((src, i) => {{
  const t = document.createElement("img");
  t.src = src;
  t.onclick = () => {{ jumpTo(i); resetTimer(); }};
  thumbs.appendChild(t);
}});

// Helper: create a linked slide
function createSlide(src, href, className) {{
  const link = document.createElement("a");
  link.href = href;
  link.target = "_blank";
  link.className = className;

  const img = document.createElement("img");
  img.src = src;

  link.appendChild(img);
  return link;
}}

// Create initial slide
let current = createSlide(images[0], urls[0], "slide center");
slideshow.appendChild(current);

function slideTo(newIndex, direction) {{
  const incoming = createSlide(
    images[newIndex],
    urls[newIndex],
    "slide " + (direction === "next" ? "off-right" : "off-left")
  );
  slideshow.appendChild(incoming);

  incoming.getBoundingClientRect();

  current.className =
    "slide " + (direction === "next" ? "off-left" : "off-right");
  incoming.className = "slide center";

  setTimeout(() => {{
    slideshow.removeChild(current);
    current = incoming;
  }}, {ANIM_MS});

  index = newIndex;
  updateCounter();
  updateThumbnails();
}}

function nextSlide() {{
  slideTo((index + 1) % images.length, "next");
  resetTimer();
}}

function prev() {{
  slideTo((index - 1 + images.length) % images.length, "prev");
  resetTimer();
}}

function jumpTo(i) {{
  if (i === index) return;
  slideTo(i, i > index ? "next" : "prev");
  resetTimer();
}}

function startAuto() {{
  timer = setInterval(() => {{
    if (auto) nextSlide();
  }}, {AUTO_ADVANCE_MS});
}}

function toggleAuto() {{
  auto = !auto;
  toggleBtn.textContent = auto ? "⏸ Pause" : "▶ Resume";
  resetTimer();
}}

// Touch swipe
let startX = 0;

slideshow.addEventListener("touchstart", e => {{
  startX = e.touches[0].clientX;
}});

slideshow.addEventListener("touchend", e => {{
  const endX = e.changedTouches[0].clientX;
  const dx = endX - startX;

  if (Math.abs(dx) > 50) {{
    if (dx < 0) nextSlide();
    else prev();
  }}
}});

// Keyboard
document.addEventListener("keydown", (e) => {{
  if (e.key === "ArrowRight") nextSlide();
  if (e.key === "ArrowLeft") prev();
  if (e.key === " ") toggleAuto();
}});

// CLICK TO ZOOM
document.addEventListener("click", (e) => {{
  if (e.target.tagName === "IMG" && e.target.closest(".slide")) {{
    e.target.classList.toggle("zoomed");
  }}
}});

updateCounter();
updateThumbnails();
startAuto();
</script>

</body>
</html>
"""

with open("index.html", "w", encoding="utf-8") as f:
    f.write(html)

print(f"Generated index.html with animated sliding transitions ({len(images)} images)")


Generated index.html with animated sliding transitions (6 images)


In [7]:
import glob

extensions = ("*.png", "*.jpg", "*.jpeg")

images = []
for ext in extensions:
    images.extend(sorted(glob.glob(ext)))

if not images:
    raise RuntimeError("No images found in current directory")

AUTO_ADVANCE_MS = 5000
ANIM_MS = 400

# Build JS arrays
images_js = "[" + ",".join(f"'{img}'" for img in images) + "]"
urls_js = "[" + ",".join("'http://meritpages.com/michael_woodcock'" for _ in images) + "]"

html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>Awards Slideshow</title>

<style>
  body {{
    font-family: Arial, sans-serif;
    background: #121212;
    color: #e0e0e0;
    margin: 0;
    overflow: hidden;
  }}

  .back-button {{
    position: fixed;
    top: 1rem;
    left: 1rem;
    display: flex;
    align-items: center;
    gap: 0.5rem;
    background: rgba(30, 30, 30, 0.9);
    border: 1px solid rgba(0, 255, 255, 0.2);
    border-radius: 999px;
    padding: 0.4rem 0.9rem;
    color: #ffffff;
    font-size: 1.1rem;
    cursor: pointer;
    z-index: 10000;
    transition: all 0.3s ease;
    box-shadow: 0 0 6px rgba(0, 255, 255, 0.3);
  }}

  .back-button:hover {{
    background: rgba(0, 255, 255, 0.15);
    color: #00bcd4;
    transform: scale(1.05);
    box-shadow: 0 0 10px rgba(0, 255, 255, 0.6);
  }}

  /* FULL-VIEWPORT SLIDESHOW */
  .slideshow {{
    position: relative;
    width: 100vw;
    height: 100vh;
    display: flex;
    align-items: center;
    justify-content: center;
    overflow: hidden;
  }}

  /* SAFE IMAGE AREA — NEVER OVERLAPS UI */
  .slide {{
    position: absolute;
    top: 0;
    left: 0;
    width: calc(100vw - 20px);      /* 10px padding left/right */
    height: calc(100vh - 120px);    /* compact UI zone */
    padding: 10px;                  /* guaranteed padding */
    box-sizing: border-box;
    display: flex;
    align-items: center;
    justify-content: center;
    transition: transform {ANIM_MS}ms ease;
  }}

  /* Image ALWAYS fits inside padded safe area */
  .slide img {{
    width: 100%;
    height: 100%;
    object-fit: contain;
    border-radius: 6px;
    box-shadow: 0 20px 60px rgba(0,0,0,0.7);
    display: block;
    cursor: pointer; /* click opens link */
    transition: transform 0.15s ease-out;
  }}

  .off-right {{ transform: translateX(100vw); }}
  .off-left  {{ transform: translateX(-100vw); }}
  .center    {{ transform: translateX(0); }}

  /* COMPACT CONTROLS */
  .controls {{
    position: fixed;
    bottom: 70px;
    width: 100%;
    display: flex;
    justify-content: center;
    gap: 10px;
    z-index: 1000;
  }}

  button.control {{
    background: #1f2937;
    color: #e5e7eb;
    border: 1px solid #374151;
    padding: 6px 12px;
    font-size: 13px;
    border-radius: 6px;
    cursor: pointer;
  }}

  .counter {{
    position: fixed;
    bottom: 50px;
    width: 100%;
    text-align: center;
    font-size: 12px;
    color: #9ca3af;
    z-index: 1000;
  }}

  /* THUMBNAILS — NO SCROLLING EVER */
  .thumb-strip {{
    position: fixed;
    bottom: 0;
    width: 100%;
    height: 50px;
    background: rgba(0,0,0,0.4);
    display: flex;
    justify-content: center;
    align-items: center;
    gap: 6px;
    padding: 4px;
    box-sizing: border-box;
    overflow: hidden; /* NO SCROLLING */
    flex-wrap: wrap;  /* wrap instead of scroll */
  }}

  .thumb-strip img {{
    height: 40px;
    border-radius: 4px;
    cursor: pointer;
    opacity: 0.6;
    transition: opacity 0.2s, transform 0.2s;
  }}

  .thumb-strip img.active {{
    opacity: 1;
    transform: scale(1.05);
    border: 2px solid cyan;
  }}
</style>
</head>

<body>

<button class="back-button" onclick="history.back()">
  <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24">
    <path d="M15 18l-6-6 6-6"/>
  </svg>
  <span>Back</span>
</button>

<div class="slideshow"></div>

<div class="controls">
  <button class="control" onclick="prev()">⟵ Previous</button>
  <button class="control" onclick="toggleAuto()" id="toggle">⏸ Pause</button>
  <button class="control" onclick="nextSlide()">Next ⟶</button>
</div>

<div class="counter" id="counter"></div>

<div class="thumb-strip" id="thumbs"></div>

<script>
// Auto-generated by Python
const images = {images_js};
const urls = {urls_js};

let index = 0;
let auto = true;
let timer = null;

const slideshow = document.querySelector(".slideshow");
const counter = document.getElementById("counter");
const toggleBtn = document.getElementById("toggle");
const thumbs = document.getElementById("thumbs");

function resetTimer() {{
  if (timer) clearInterval(timer);
  startAuto();
}}

function updateCounter() {{
  counter.textContent = `${{index + 1}} / ${{images.length}}`;
}}

function updateThumbnails() {{
  [...thumbs.children].forEach((t, i) => {{
    t.classList.toggle("active", i === index);
  }});
}}

// Build thumbnails
images.forEach((src, i) => {{
  const t = document.createElement("img");
  t.src = src;
  t.onclick = () => {{ jumpTo(i); resetTimer(); }};
  thumbs.appendChild(t);
}});

// Helper: create a linked slide
function createSlide(src, href, className) {{
  const link = document.createElement("a");
  link.href = href;
  link.target = "_blank";
  link.className = className;

  const img = document.createElement("img");
  img.src = src;

  link.appendChild(img);
  return link;
}}

// Create initial slide
let current = createSlide(images[0], urls[0], "slide center");
slideshow.appendChild(current);

function slideTo(newIndex, direction) {{
  const incoming = createSlide(
    images[newIndex],
    urls[newIndex],
    "slide " + (direction === "next" ? "off-right" : "off-left")
  );
  slideshow.appendChild(incoming);

  incoming.getBoundingClientRect();

  current.className =
    "slide " + (direction === "next" ? "off-left" : "off-right");
  incoming.className = "slide center";

  setTimeout(() => {{
    slideshow.removeChild(current);
    current = incoming;
  }}, {ANIM_MS});

  index = newIndex;
  updateCounter();
  updateThumbnails();
}}

function nextSlide() {{
  slideTo((index + 1) % images.length, "next");
  resetTimer();
}}

function prev() {{
  slideTo((index - 1 + images.length) % images.length, "prev");
  resetTimer();
}}

function jumpTo(i) {{
  if (i === index) return;
  slideTo(i, i > index ? "next" : "prev");
  resetTimer();
}}

function startAuto() {{
  timer = setInterval(() => {{
    if (auto) nextSlide();
  }}, {AUTO_ADVANCE_MS});
}}

function toggleAuto() {{
  auto = !auto;
  toggleBtn.textContent = auto ? "⏸ Pause" : "▶ Resume";
  resetTimer();
}}

// Touch swipe
let startX = 0;

slideshow.addEventListener("touchstart", e => {{
  startX = e.touches[0].clientX;
}});

slideshow.addEventListener("touchend", e => {{
  const endX = e.changedTouches[0].clientX;
  const dx = endX - startX;

  if (Math.abs(dx) > 50) {{
    if (dx < 0) nextSlide();
    else prev();
  }}
}});

// Keyboard
document.addEventListener("keydown", (e) => {{
  if (e.key === "ArrowRight") nextSlide();
  if (e.key === "ArrowLeft") prev();
  if (e.key === " ") toggleAuto();
}});

// SCROLL WHEEL ZOOM
let zoomLevel = 1;

slideshow.addEventListener("wheel", (e) => {{
  e.preventDefault();

  const img = current.querySelector("img");
  if (!img) return;

  if (e.deltaY < 0) zoomLevel += 0.1;   // zoom in
  else zoomLevel -= 0.1;               // zoom out

  zoomLevel = Math.max(1, Math.min(3, zoomLevel));

  img.style.transform = `scale(${{zoomLevel}})`;
}}, {{ passive: false }});

updateCounter();
updateThumbnails();
startAuto();
</script>

</body>
</html>
"""

with open("index.html", "w", encoding="utf-8") as f:
    f.write(html)

print(f"Generated index.html with animated sliding transitions ({len(images)} images)")


Generated index.html with animated sliding transitions (6 images)
